In [13]:
import pandas as pd
import os
import re
from sklearn.preprocessing import LabelEncoder

In [14]:
raw_data = pd.read_csv('data\\df_clear_uniq_3.csv')
print(raw_data.describe())
raw_data.head()

               part_num       manufacturer       TNVED  \
count             12346              12346       12346   
unique            12076                698          87   
top     AT90CAN128-16AU  TEXAS INSTRUMENTS  8542399010   
freq                  3                995        2676   

                      concat_str  \
count                      12346   
unique                     12342   
top     KB-H15-25PN-XP KINGBRITE   
freq                           2   

                                                  path_ds  \
count                                               12346   
unique                                              12346   
top     M:\DataSheet\d76e3861-23b6-11ed-96e5-f8bc12505...   
freq                                                    1   

                                     id_path  
count                                  12346  
unique                                 12346  
top     d76e3861-23b6-11ed-96e5-f8bc12505841  
freq                              

,part_num,manufacturer,TNVED,concat_str,path_ds,id_path
0,XC7K410T-2FFG900I,XILINX,8542399010,XC7K410T-2FFG900I XILINX,M:\DataSheet\d76e3861-23b6-11ed-96e5-f8bc12505...,d76e3861-23b6-11ed-96e5-f8bc12505841
1,NC7WZ04P6X,ON SEMICONDUCTOR,8542399010,NC7WZ04P6X ON SEMICONDUCTOR,M:\DataSheet\b09bd526-4a32-11ed-96e6-f8bc12505...,b09bd526-4a32-11ed-96e6-f8bc12505841
2,ADUM1401ARWZ,ANALOG DEVICES,8542399010,ADUM1401ARWZ ANALOG DEVICES,M:\DataSheet\2ff0bfb5-6804-11ee-96f4-f8bc12505...,2ff0bfb5-6804-11ee-96f4-f8bc12505841
3,GCM155R71C104KA55D,MURATA ELECTRONICS,8532240000,GCM155R71C104KA55D MURATA ELECTRONICS,M:\DataSheet\8436bcbb-4a18-11ed-96e6-f8bc12505...,8436bcbb-4a18-11ed-96e6-f8bc12505841
4,ADA4898-1YRDZ,ANALOG DEVICES,8542339000,ADA4898-1YRDZ ANALOG DEVICES,M:\DataSheet\cb8a2892-ddc2-11ed-96f0-f8bc12505...,cb8a2892-ddc2-11ed-96f0-f8bc12505841


Пропишем путь до даташитов корректно

Посмотрим сколько всего битых строк, которые ссылаются на несуществующие даташиты.


In [15]:
datasheet_path = 'data\\datasheets'

raw_data['path_ds'] = raw_data['id_path'].apply(lambda x: os.path.join(datasheet_path, f'{x}.txt'))
print(f'path_ds example: {raw_data['path_ds'][0]}')  # проверяем что путь рапарсился как ожидается
raw_data['file_exists'] = raw_data['path_ds'].apply(lambda x: os.path.exists(x))
raw_data['file_exists'].value_counts()


path_ds example: data\datasheets\d76e3861-23b6-11ed-96e5-f8bc12505841.txt


file_exists
True     10152
False     2194
Name: count, dtype: int64

около 17% битых строк - удаляем. Без суммаризации они бесполезны

In [16]:
raw_data = raw_data.drop(raw_data[~raw_data['file_exists']].index)
raw_data = raw_data.drop(columns=['file_exists'])
raw_data.describe()

,part_num,manufacturer,TNVED,concat_str,path_ds,id_path
count,10152,10152,10152,10152,10152,10152
unique,9963,632,85,10149,10152,10152
top,1.5SMC6.8CA,TEXAS INSTRUMENTS,8542399010,HY820M400Y350A00T CAPXON,data\datasheets\d76e3861-23b6-11ed-96e5-f8bc12...,d76e3861-23b6-11ed-96e5-f8bc12505841
freq,3,869,2247,2,1,1


Удалим класс other без ТН ВЭД кода

In [17]:
raw_data = raw_data.drop(raw_data[raw_data['TNVED']=='other'].index)


Регулярное выражение ниже ищет слова из списка:
- Description
- Component Description
- Description of the Component
- General Description
- Product Description.

проверяет что следущий за ключевым словом текст начинается с The(the)

захватывает весь текст до следующих ключевых слов:
- Key Specifications
- Typical Applications


In [18]:
def extract_description(text):
    # удаление специальных символов (#, *, \b)
    cleaned_text = re.sub(r"[#\*\b]", "", text)
    
    
    pattern = re.compile(
        r"(?i)(?:Description|Component Description|Description of the Component|General Description|Product Description)\s*:?\s*([tT]he[\s\S]*?)(?=Key Specifications|Typical Applications|$)",
        re.IGNORECASE
    )
    
   
    match = pattern.search(cleaned_text)
    if match:
        return match.group(1).strip()  # возвращаем захваченный текст
    return None

def read_summ(file_path):
    try:
        with open(file_path, encoding='utf-8') as file:
            text = file.read()
        description = extract_description(text)
        return description
    except Exception as exc:
        return None
  
    

test = raw_data['path_ds'][173]
description = read_summ(test)
print(description)



The Aliner 20-001A0-T is a precision SMA straight jack connector designed for microwave applications. It features a robust construction with gold plating for enhanced conductivity and corrosion resistance. This connector is ideal for high-frequency signal transmission, making it suitable for various RF and microwave applications.


In [19]:
raw_data['description'] = (raw_data['path_ds']).apply(read_summ)

Те 60 None, которые остались это нерелевантный ответ gpt по типу: "а сделай суммаризацию сам, вот тебе инструкция".

Их удаляем

In [20]:
print(raw_data['description'].isna().value_counts())
print(raw_data[raw_data['description'].isna()]['id_path'])
raw_data = raw_data.dropna(subset=['description'])

description
False    9920
True       44
Name: count, dtype: int64
1623     ef85cbcf-905f-11ee-96f7-f8bc12505841
3142     4e965255-2332-11ef-96f9-f8bc12505841
3237     93b98ab7-d9f8-11ee-96f7-f8bc12505841
3634     773b2750-2265-11ef-96f9-f8bc12505841
3708     3f742ca0-10f4-11ef-96f7-f8bc12505841
3720     68b65ac3-021b-11ef-96f7-f8bc12505841
3721     4ef5639a-021b-11ef-96f7-f8bc12505841
5202     91dd3ba0-e200-11ee-96f7-f8bc12505841
6038     e4fc7d2a-ebf0-11ec-96e1-f8bc12505841
6047     75c987ce-a0cb-11ee-96f7-f8bc12505841
6048     75c98776-a0cb-11ee-96f7-f8bc12505841
6147     d7eb8b4a-5228-11ee-96f3-f8bc12505841
6148     104ca5a7-52e2-11ee-96f4-f8bc12505841
6756     43a477de-97f4-11ee-96f7-f8bc12505841
6935     54c56300-34ea-11ed-96e6-f8bc12505841
7296     abb63433-5d25-11ee-96f4-f8bc12505841
7360     d2a4487d-a581-11ee-96f7-f8bc12505841
7637     f614fc02-7257-11ee-96f6-f8bc12505841
7995     60850ca7-e06a-11ee-96f7-f8bc12505841
8220     5bd0d87d-2ae8-11ee-96f2-f8bc12505841
8270     e2093

Объединяем в один текстовый признак название, производителя и описание

In [21]:
raw_data['text'] = raw_data['concat_str'] + ' ' + raw_data['description']

Энкодируем таргет

In [22]:

raw_data['TNVED_encoded'] = LabelEncoder().fit_transform(raw_data['TNVED'])

Сохраняем конечный csv для дальнейшего обучения

In [23]:
data = raw_data[['text', 'TNVED_encoded']]
data.to_csv('./data/data.csv', index=False)